## Information is not lost for simple maze network

In [1]:
# import the required packages
from PIL import Image
import numpy as np

# convert the maze image to grayscale
image = Image.open('test2.jpg').convert('L')
w, h = image.size # get the dimenssions

# resize every image to a fixed shape
#image = image.resize((100,100))

# set all white pixels to 1 and black to 0
binary = image.point(lambda p: p > 128 and True)  # 0 is blocked and 1 is unblocked

# Resize the binary so that we can reduce lot of 0's and 1's to traverse - this value is hardcoded for now
# we need to figure out a way to set w,h value without lossing information
binary = binary.resize((1300,1300),Image.NEAREST)
w, h = binary.size

# converting image to numpy array
nim = np.array(binary)
print(nim.shape)
'''for r in range(100):
    for c in range(w):
        print(nim[r,c],end='')
    print()
for r in range(100):
    for c in range(w):
        if(nim[r,c]==1):
            print(r,c)
print(w,h)'''

(1300, 1300)


"for r in range(100):\n    for c in range(w):\n        print(nim[r,c],end='')\n    print()\nfor r in range(100):\n    for c in range(w):\n        if(nim[r,c]==1):\n            print(r,c)\nprint(w,h)"

## Information is lost for a complex maze

In [2]:
import sys
from PIL import Image
import numpy
import cv2

def AStar(start, goal, neighbor_nodes, distance, cost_estimate):
    def reconstruct_path(came_from, current_node):
        path = []
        while current_node is not None:
            path.append(current_node)
            current_node = came_from[current_node]
        return list(reversed(path))

    g_score = {start: 0}
    f_score = {start: g_score[start] + cost_estimate(start, goal)}
    openset = {start}
    closedset = set()
    came_from = {start: None}

    while openset:
        current = min(openset, key=lambda x: f_score[x])
        if current == goal:
            return reconstruct_path(came_from, goal)
        openset.remove(current)
        closedset.add(current)
        for neighbor in neighbor_nodes(current):
            if neighbor in closedset:
                continue
            if neighbor not in openset:
                openset.add(neighbor)
            tentative_g_score = g_score[current] + distance(current, neighbor)
            if tentative_g_score >= g_score.get(neighbor, float('inf')):
                continue
            came_from[neighbor] = current
            g_score[neighbor] = tentative_g_score
            f_score[neighbor] = tentative_g_score + cost_estimate(neighbor, goal)
    return []


def is_blocked(p):
    x,y = p
    pixel = nim[x,y]
    if (pixel == 0):
        return True

def von_neumann_neighbors(p):
    x, y = p
    neighbors = [(x-1, y), (x, y-1), (x+1, y), (x, y+1)]
    return [p for p in neighbors if not is_blocked(p)]

def manhattan(p1, p2):
    return abs(p1[0]-p2[0]) + abs(p1[1]-p2[1])

def squared_euclidean(p1, p2):
    return (p1[0]-p2[0])**2 + (p1[1]-p2[1])**2

#3
# start = (400, 984)
# goal = (398, 25)

#test 1
#start = (0,25)
#goal = (98,51)

#4
# Took values since exact values are producing index out of bound error while traversing neighbors

start = (1,1)
goal = (1298,30)

# test 3
#start =(0,0)
#stop = ()
# invoke: python mazesolver.py <mazefile> <outputfile>[.jpg|.png|etc.]

distance = squared_euclidean
heuristic = squared_euclidean

# 9 path is the solution path i dont know why there is another path in between should try different algo for path maybe.
path = AStar(start, goal, von_neumann_neighbors, distance, heuristic)
for position in path:
    x,y = position
    nim[x,y] = 9
    


In [3]:
for r in range(h):
    for c in range(w):
        i=1
        j=1
        if nim[r,c] == 9:
            # result[r,c,:] = (0,255,0)
            while(nim[r,c+i] == 1 and i<=5 and c+i<w):
                nim[r,c+i] =2
                i=i+1
            while(nim[r,c-i] == 1 and 10-i>=0 and c-i>w):
                nim[r,c-i] = 2
                i=i+1
            while(nim[r+j,c] == 1 and 10-j>=0 and c-j>h):
                nim[r+j,c] = 2
                j=j+1
            while(nim[r-j,c] == 1 and 10-j>=0 and c-j>h):
                nim[r-j,c] = 2
                j=j+1

In [5]:
result = numpy.ones([1300,1300,3])

for r in range(h):
    for c in range(w):
        if nim[r,c] == 1:
            result[r,c,:] = (255,255,255)
        elif nim[r,c] == 0:
            result[r,c,:] = (0,0,0)

In [6]:
for r in range(h):
    for c in range(w):
        if nim[r,c] == 2 or nim[r,c] == 9:
            result[r,c,:] = (0,255,0)
    cv2.imshow("result",result)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
cv2.destroyAllWindows()

In [7]:
import cv2
cv2.imwrite("result.jpg", result)

True

In [ ]:
for r in range(h):
    for c in range(w):
        print(nim[r,c],end='')
    print()